<a href="https://colab.research.google.com/github/lyuoveta/git_test_model_1000/blob/master/Llama/%20Llama_2%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 5.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 42.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 9.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-bejuhvim/overlay/local/bin
    changing mode of /tmp/pip-build-env-bejuhvim/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-bejuhvim/overlay/l

In [7]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [8]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [9]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [10]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [11]:

from google.colab import files


pd= files.upload()

Saving tagged_data_llama_256_1.csv to tagged_data_llama_256_1.csv


In [12]:
import pandas as pd
import random
df = pd.read_csv('tagged_data_llama_256_1.csv')


In [13]:
df = df[['reviews', 'Correct review ', 'Incorrect review ']]
df.head()

,reviews,Correct review,Incorrect review
0,Not bad but could be better,1,0
1,Taking biotin 5000mcg from October first week ...,1,0
2,My nails grew in noticeably thicker and strong...,1,0
3,I took biotin for 1 week. I started having mu...,1,0
4,I used to have very thin nails that bent when ...,1,0


In [14]:
df_incorrect = df.loc[df['Incorrect review '] == 1]['reviews'].reset_index(drop=True)
df_correct = df.loc[df['Correct review '] == 1]['reviews'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'This stuff is Great! I take 2 a day and my nails grow faster and they are thicker. I Love this stuff and my hair grows faster and is not falling out. It almost looks like some of my grey hair is getting color back in it.'

In [15]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'Would there possibly be a skin rash side effect?'

In [ ]:
#Step 6: Generating the Response

In [16]:
for _ in range(20):
    prompt = f'''Please evaluate the following review for correctness. A '1' indicates that the "
            "review is correct (it is not spam and contains information about the product effects), "
            "while a '0' indicates an incorrect review (it is spam or doesn't contain information about the product effects):
    1. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    2. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    3. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    4. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    5. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1

    Please provide the correctness only of the following review by entering 0 for incorrect or 1 for correct:
    "{df['reviews'][random.choice(range(len(df)))]}"
    '''
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.

    USER: {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.2, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)
    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    matches = re.findall(r'ASSISTANT:.*?([01])', response_text, re.DOTALL)

    print(matches)

{'id': 'cmpl-2b8adf86-786c-49d4-a045-fd662378bbe8', 'object': 'text_completion', 'created': 1701611898, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I feel like Melatonin helps me fall to sleep easier, but I still have trouble staying asleep.\n     ANSWER: 1\n    2. I\'m 57 years old. I\'d swear when I take Ginko Biloba at double or triple the recommended dose I get frequent nocturnal erections, especially towards morning.\n     AN

Llama.generate: prefix-match hit


{'id': 'cmpl-8e0c12a9-6ed3-4474-9da7-7e0683dba2db', 'object': 'text_completion', 'created': 1701611951, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. Less neck and shoulder pain and stiffness than before using it.\n     ANSWER: 1\n    2. I take orcerin GM also with bioD3pls\n     ANSWER: 0\n    3. I take folic acid to stop  hot flushes due to menopause . Reduced them tremendously and returned with vengeance when I stopped taking the

Llama.generate: prefix-match hit


{'id': 'cmpl-96853479-728c-416a-8eb8-591d58713d14', 'object': 'text_completion', 'created': 1701612031, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. My hair has started to grow in after 3 months and I could not be happier with the product. Could not believe the results and my hairdresser is also happy with the results.\n     ANSWER: 1\n    2. wonderfull\n     ANSWER: 0\n    3. Cures hemmoroids within days.\n     ANSWER: 1\n    4. N

...

Llama.generate: prefix-match hit


{'id': 'cmpl-183e2d34-f149-4716-bdd5-ba8ca8464bec', 'object': 'text_completion', 'created': 1701612752, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. having abdomen pain after taking melatonin\n     ANSWER: 1\n    2. Basically it does exactly what metformin does only without the side effects. Also has further benefits metformin doesn\'t. Big pharma does not want you knowing this. Non for profit medical system countrues regularly pre

Llama.generate: prefix-match hit


{'id': 'cmpl-96bd2c74-62a9-4567-a692-a809e2970d4f', 'object': 'text_completion', 'created': 1701612787, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. With a number of pain-causing senior-years issues, melatonin is suggested by my general practitioner to help me get more sleep.  It doesn\'t appear to help, and I am lucky to get 5 hrs./night sleep.  Not enough!\n     ANSWER: 1\n    2. just started taking 10,400i/u\'s a day 6/04/2010\n

ValueError: ignored

In [17]:
!pip install scikit-learn

true_labels = [1,1,0,1,0,0,1,1,1,1]
predicted_labels = [0,1,1,0,0,1,1,1,1,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")


Точность модели: 60.00%
